In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Example") \
    .config("spark.sql.warehouse.dir", "/spark-warehouse") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# Test the connection by running a simple query.
databases = spark.sql("SHOW DATABASES")
databases.show()


2025-03-20 01:56:58 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+------------+
|databaseName|
+------------+
|   cancer_db|
|     default|
+------------+



In [2]:
spark.sql("CREATE DATABASE IF NOT EXISTS cancer_db")

DataFrame[]

In [3]:
spark.sql("SHOW DATABASES").show()


+------------+
|databaseName|
+------------+
|   cancer_db|
|     default|
+------------+



In [4]:
!ls -ld /spark-warehouse

drwxrwxrwx 3 root root 4096 Mar 20 01:52 /spark-warehouse


In [5]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/data/Cancer.csv")


In [6]:
# df.printSchema()
df.show(5)


+----------+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+---------------------+------------------------+-------------+---------+-------+------+
|Patient Id|Age|Gender|Air Pollution|Alcohol use|Dust Allergy|OccuPational Hazards|Genetic Risk|chronic Lung Disease|Balanced Diet|Obesity|Smoking|Passive Smoker|Chest Pain|Coughing of Blood|Fatigue|Weight Loss|Shortness of Breath|Wheezing|Swallowing Difficulty|Clubbing of Finger Nails|Frequent Cold|Dry Cough|Snoring| Level|
+----------+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+---------------------+------------------------+-------------+---------+-------+------+
|        P1| 33|   

In [7]:
spark.conf.get("spark.sql.warehouse.dir")


'/spark-warehouse'

In [8]:
new_columns = [c.replace(' ', '_') for c in df.columns]
df = df.toDF(*new_columns)

# Drop existing table if exists
spark.sql("DROP TABLE IF EXISTS cancer_db.cancer_table")

# Clean column names
new_columns = [c.replace(' ', '_') for c in df.columns]
df = df.toDF(*new_columns)

# Write to table
df.write \
    .mode("overwrite") \
    .option("path", "/spark-warehouse/cancer_db.db/cancer_table") \
    .saveAsTable("cancer_db.cancer_table")

# Verify table
spark.sql("SHOW TABLES IN cancer_db").show()

# Read back the table
df_read = spark.sql("SELECT * FROM cancer_db.cancer_table")
df_read.show()


+---------+------------+-----------+
| database|   tableName|isTemporary|
+---------+------------+-----------+
|cancer_db|cancer_table|      false|
+---------+------------+-----------+

+----------+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+---------------------+------------------------+-------------+---------+-------+------+
|Patient_Id|Age|Gender|Air_Pollution|Alcohol_use|Dust_Allergy|OccuPational_Hazards|Genetic_Risk|chronic_Lung_Disease|Balanced_Diet|Obesity|Smoking|Passive_Smoker|Chest_Pain|Coughing_of_Blood|Fatigue|Weight_Loss|Shortness_of_Breath|Wheezing|Swallowing_Difficulty|Clubbing_of_Finger_Nails|Frequent_Cold|Dry_Cough|Snoring| Level|
+----------+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+-------------